In [2]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import shutil
import time

# path

In [4]:
is_debug=True
dataset_root_path='/home/ubuntu/Dataset/AD/'
pre_dataset_path = '/home/ubuntu/Dataset/preprocessing10/'
subject_tot_num = 194#len(os.listdir(dataset_root_path)) # 178(2020-02-27 기준)

In [5]:
# create directory
def make_folder(pre_dataset_path):
    '''
    
    pre_dataset_path    - S001  - E1    
    
    
                                - E2
                                - ...
                        - S002
                        - ...
    '''
    if pre_dataset_path.split('/')[-2] not in os.listdir('/home/ubuntu/Dataset/'):
        os.mkdir(pre_dataset_path)
        for subject_num in range(1,subject_tot_num+1):
            S_tmp = '/S{:03}/'.format(subject_num)
            os.mkdir(pre_dataset_path+S_tmp)
            for E_num in range(1,8):
                E_tmp = 'E{}/'.format(E_num)
                os.mkdir(pre_dataset_path+S_tmp+E_tmp)
    else:
        print('already existed')

In [6]:
make_folder(pre_dataset_path)

In [7]:
# search all of csv file
def search_csv(subject_num,comment=False):

    filename_type = ['LegL_DDCC_Calibrated_PC', 'LegR_DC90_Calibrated_PC' ,'ThighL_DC74_Calibrated_PC','ThighR_DC77_Calibrated_PC',
             'ToeL_DCD4_Calibrated_PC', 'ToeR_DCAF_Calibrated_PC' ,'Waist_E0E6_Calibrated_PC' ]

    type_dict=dict()
    for f in filename_type:
        simple = f.split('_')[0]
        type_dict[simple]=f
    res_list = []
    subfolder_name = 'S{:03}'.format(subject_num)
    if subfolder_name not in os.listdir(dataset_root_path):
        print('Not existed')
        return res_list
    else:
        sub_root_path = dataset_root_path + subfolder_name + '/'
        for (path, dir, files) in os.walk(sub_root_path):
            
            if '_raw' in path :
                # print(path)
                continue
            
            
            for filename in files:  
                if 'Waist' in filename :
                    
                    E_num ,Session_num = find_num_from_filename(filename)
                    
                    Waist_filename = path+'/'+filename
                    LegL_filename = Waist_filename.replace(type_dict['Waist'],type_dict['LegL'])
                    LegR_filename = Waist_filename.replace(type_dict['Waist'],type_dict['LegR'])
                    ThighL_filename = Waist_filename.replace(type_dict['Waist'],type_dict['ThighL'])
                    ThighR_filename = Waist_filename.replace(type_dict['Waist'],type_dict['ThighR'])
                    ToeL_filename = Waist_filename.replace(type_dict['Waist'],type_dict['ToeL'])
                    ToeR_filename = Waist_filename.replace(type_dict['Waist'],type_dict['ToeR'])
                    
                    tmp = [E_num,Waist_filename,LegL_filename,LegR_filename,ThighL_filename,ThighR_filename,ToeL_filename,ToeR_filename]

                    res_list.append(tmp)
                    
    return res_list
#    find specific e_num csv file

def find_num_from_filename(filename):

    E_num = filename[filename.find('E')+0:filename.find('E')+2] 
    Session_num = filename[filename.find('_Session')+1:filename.find('_Session')+9]
    return E_num,Session_num

In [8]:
search_csv(3)

[['E1',
  '/home/ubuntu/Dataset/AD/S003/Wearable/2017-01-16_15.02.06_0032E1_MultiSession/0032E1_Session1_Waist_E0E6_Calibrated_PC.csv',
  '/home/ubuntu/Dataset/AD/S003/Wearable/2017-01-16_15.02.06_0032E1_MultiSession/0032E1_Session1_LegL_DDCC_Calibrated_PC.csv',
  '/home/ubuntu/Dataset/AD/S003/Wearable/2017-01-16_15.02.06_0032E1_MultiSession/0032E1_Session1_LegR_DC90_Calibrated_PC.csv',
  '/home/ubuntu/Dataset/AD/S003/Wearable/2017-01-16_15.02.06_0032E1_MultiSession/0032E1_Session1_ThighL_DC74_Calibrated_PC.csv',
  '/home/ubuntu/Dataset/AD/S003/Wearable/2017-01-16_15.02.06_0032E1_MultiSession/0032E1_Session1_ThighR_DC77_Calibrated_PC.csv',
  '/home/ubuntu/Dataset/AD/S003/Wearable/2017-01-16_15.02.06_0032E1_MultiSession/0032E1_Session1_ToeL_DCD4_Calibrated_PC.csv',
  '/home/ubuntu/Dataset/AD/S003/Wearable/2017-01-16_15.02.06_0032E1_MultiSession/0032E1_Session1_ToeR_DCAF_Calibrated_PC.csv'],
 ['E1',
  '/home/ubuntu/Dataset/AD/S003/Wearable/2017-01-16_15.02.06_0032E1_MultiSession/0032E1_S

In [9]:
# create s_num and filenames dictionary 

def subject_dict():
    
    
    subject_dict = dict()
    for subject_num in range(1,subject_tot_num+1):
        S_tmp = 'S{:03}'.format(subject_num)
        tmp_dict=dict()
        for i in range(1,8):
            E_tmp = 'E{}'.format(i)
            tmp_dict[E_tmp]=[]


        search = search_csv(subject_num)
        for row in search_csv(subject_num):
            tmp_dict[row[0]].append(row[1:])


        subject_dict[S_tmp]=tmp_dict
    return subject_dict

In [10]:
subject_dict()['S003']

{'E1': [['/home/ubuntu/Dataset/AD/S003/Wearable/2017-01-16_15.02.06_0032E1_MultiSession/0032E1_Session1_Waist_E0E6_Calibrated_PC.csv',
   '/home/ubuntu/Dataset/AD/S003/Wearable/2017-01-16_15.02.06_0032E1_MultiSession/0032E1_Session1_LegL_DDCC_Calibrated_PC.csv',
   '/home/ubuntu/Dataset/AD/S003/Wearable/2017-01-16_15.02.06_0032E1_MultiSession/0032E1_Session1_LegR_DC90_Calibrated_PC.csv',
   '/home/ubuntu/Dataset/AD/S003/Wearable/2017-01-16_15.02.06_0032E1_MultiSession/0032E1_Session1_ThighL_DC74_Calibrated_PC.csv',
   '/home/ubuntu/Dataset/AD/S003/Wearable/2017-01-16_15.02.06_0032E1_MultiSession/0032E1_Session1_ThighR_DC77_Calibrated_PC.csv',
   '/home/ubuntu/Dataset/AD/S003/Wearable/2017-01-16_15.02.06_0032E1_MultiSession/0032E1_Session1_ToeL_DCD4_Calibrated_PC.csv',
   '/home/ubuntu/Dataset/AD/S003/Wearable/2017-01-16_15.02.06_0032E1_MultiSession/0032E1_Session1_ToeR_DCAF_Calibrated_PC.csv'],
  ['/home/ubuntu/Dataset/AD/S003/Wearable/2017-01-16_15.02.06_0032E1_MultiSession/0032E1_Ses

In [12]:
# select coloumn from csv file 
def sel_col(df, col_name):
    res = ''
    for column in df.columns:
        if col_name in column:
            res = column
            break
    if res == '':
        return
#         print('not existed')
    else:
        return res 
# load csv file
def load_data(filename):
    df = pd.read_csv(filename , skiprows=[0,2], sep='	')
    col_list = ['Timestamp', 'Accel_WR_X','Accel_WR_Y','Accel_WR_Z' , 'Gyro_X','Gyro_Y','Gyro_Z' , 'Mag_X','Mag_Y','Mag_Z','Marker']
    
    data_dict= dict()
    
    for col in col_list :
        data_dict[sel_col(df,col)] = list(df[sel_col(df,col)])

    
    
    try:
        T1_stind = data_dict[sel_col(df,'Marker')].index(2)
        T1_endind= T1_stind + data_dict[sel_col(df,'Marker')][T1_stind:].index(0) -1
    except:
        T1_stind,T1_endind = None,None
    
    try:
        T2_stind= T1_endind +1 + data_dict[sel_col(df,'Marker')][T1_endind+1:].index(2) 
        T2_endind= T2_stind + data_dict[sel_col(df,'Marker')][T2_stind:].index(0) -1
    except:
        T2_stind,T2_endind = None,None
    
    data_dict['pulse']=[T1_stind,T1_endind,T2_stind,T2_endind]
    return data_dict

In [13]:
# Pre-processing according to s_num, e_num
def preprocessing(sbd,subject_num, e_num):
    S_tmp  = 'S{:03}'.format(subject_num)
    E_tmp = 'E{}'.format(e_num)
    
    
#     Fs = 64;
#     Fc_lp = 6
#     Blp,Alp = signal.butter(8,Fc_lp/(Fs/2))
    
    
    
    for index,[Waist_filename,LegL_filename,LegR_filename,ThighL_filename,ThighR_filename,ToeL_filename,ToeR_filename] in  enumerate(sbd[S_tmp][E_tmp]):
        
        try:
            Waist_data = load_data(Waist_filename)
            LegL_data,LegR_data = load_data(LegL_filename),load_data(LegR_filename)
            ToeL_data,ToeR_data = load_data(ToeL_filename),load_data(ToeR_filename)
            ThighL_data = load_data(ThighL_filename)
            try:
                ThighR_data = load_data(ThighR_filename)
            except:
                tmp_filename = ThighR_filename.replace('ThighR_DC77_Calibrated_PC','ThighR2_A640_Calibrated_PC')
                ThighR_data = load_data(tmp_filename)
        except:
            return
            
#         data_list = [LegL_data,LegR_data]
        data_list = [Waist_data,LegL_data,LegR_data,ThighL_data,ThighR_data,ToeL_data,ToeR_data]
        T1_len_min = 9999
        T2_len_min = 9999
        
        for data in data_list:
            T1_stind,T1_endind,T2_stind,T2_endind = data['pulse'][0],data['pulse'][1],data['pulse'][2],data['pulse'][3]
#             if T2_stind is not None:
#                 print(T1_stind,T1_endind,T2_stind,T2_endind)
            try:
                p_len = T1_endind - T1_stind +1
                if p_len < T1_len_min:
                    T1_len_min = p_len
            except:
                pass
            
            try:
                p_len = T2_endind - T2_stind +1
                
                if p_len < T2_len_min:
                    T2_len_min = p_len
                
            except:
                pass
        
        T1_res,T1_len = [],[]
        T2_res,T2_len = [],[]
        for i,data in enumerate(data_list):
#             print(i)
            T1_stind,T1_endind,T2_stind,T2_endind = data['pulse'][0],data['pulse'][1],data['pulse'][2],data['pulse'][3]
            
            if T1_stind is not None:

                for col in data:
                    if 'TimestampSync' not in col and 'Event' not in col and 'pulse' not in col :
#                         print(col)
                        tmp = data[col][T1_stind:T1_endind+1]
#                         tmp = signal.filtfilt(Blp,Alp,tmp, padlen=3*(max(len(Blp),len(Alp))-1))
                        f = np.array(tmp)
                        T1_res.append(f)
                        T1_len.append(np.shape(f)[0])
                
                T1_mean_len = int(np.mean(T1_len))
                
                for i,ts in enumerate(T1_res):
                    T1_res[i] = signal.resample(ts,T1_mean_len)
            if T2_stind is not None:

                for col in data:
                    if 'TimestampSync' not in col and 'Event' not in col and 'pulse' not in col :
                        tmp = data[col][T2_stind:T2_endind+1]
                        f = np.array(tmp)
                        T2_res.append(f)
                        T2_len.append(np.shape(f)[0])
                T2_mean_len = int(np.mean(T2_len))
                
                for i,ts in enumerate(T2_res):
                    T2_res[i] = signal.resample(ts,T2_mean_len)
            
#         print(np.shape(T1_res))
#         print(np.shape(T2_res))
            
                
#         save_path = pre_dataset_path + '/' + S_tmp + '/' + E_tmp + '/' + str(index)
#         print(np.shape(T1_res))
#         print(np.shape(T2_res))
        if len(T1_res) is 63:
#             if np.shape(T1_res)[1] is 256 :
            save_path = pre_dataset_path + '/' + S_tmp + '/' + E_tmp + '/' + '{}_T1'.format(index)
            np.save(save_path,T1_res)
#         else:
#                 print('T1 len',np.shape(T1_res)[1])
            
        if len(T2_res) is 63:
#             if np.shape(T2_res)[1] is 256 :
            save_path = pre_dataset_path + '/' + S_tmp + '/' + E_tmp + '/' + '{}_T2'.format(index)
            np.save(save_path,T2_res)
#         else:
#             print('T2 len')#,np.shape(T2_res)[1])
#         else:
#             print(np.shape(T2_res))
#         print(index,np.shape(T1_res),np.shape(T2_res))
        
    return 

In [14]:
# main function
def main():
    # E1~E4 session data .npy file create 
    sbd = subject_dict()
    # shutil.rmtree(pre_dataset_path)
    make_folder(pre_dataset_path)
    for subject_num in range(1,subject_tot_num+1):
        print('S{:03}'.format(subject_num))
        for e_num in range(1,5):
            preprocessing(sbd,subject_num,e_num)


In [15]:

main()

already existed
S001
S002
S003
S004
S005
S006
S007
S008
S009
S010
S011
S012
S013
S014
S015
S016
S017
S018
S019
S020
S021
S022
S023
S024
S025
S026
S027
S028
S029
S030
S031
S032
S033
S034
S035
S036
S037
S038
S039
S040
S041
S042
S043
S044
S045
S046
S047
S048
S049
S050
S051
S052
S053
S054
S055
S056
S057
S058
S059
S060
S061
S062
S063
S064
S065
S066
S067
S068
S069
S070
S071
S072
S073
S074
S075
S076
S077
S078
S079
S080
S081
S082
S083
S084
S085
S086
S087
S088
S089
S090
S091
S092
S093
S094
S095
S096
S097
S098
S099
S100
S101
S102
S103
S104
S105
S106
S107
S108
S109
S110
S111
S112
S113
S114
S115
S116
S117
S118
S119
S120
S121
S122
S123
S124
S125
S126
S127
S128
S129
S130
S131
S132
S133
S134
S135
S136
S137
S138
S139
S140
S141
S142
S143
S144
S145
S146
S147
S148
S149
S150
S151
S152
S153
S154
S155
S156
S157
S158
S159
S160
S161
S162
S163
S164
S165
S166
S167
S168
S169
S170
S171
S172
S173
S174
S175
S176
S177
S178
S179
S180
S181
S182
S183
S184
S185
S186
S187
S188
S189
S190
S191
S192
S193
S194
